In [1]:
import ee
import sys
sys.path.insert(0, '..')
import RadGEEToolbox.LandsatCollection as LandsatCollection
import RadGEEToolbox.Sentinel2Collection as Sentinel2Collection
import RadGEEToolbox.GetPalette
import RadGEEToolbox.VisParams

In [2]:
ee.Initialize()

### Defining collections - required arguments are start and end dates, and either tile(s), boundary/geometry, or relative orbit(s) (orbits for Sentinel-2 only)

In [3]:
#General way to define a Sentinel-2 image collection using start date, end date, and a single MGRS tile
S2_col = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', tile='12TUL')

#General way to define a Sentinel-2 image collection using start date, end date, and multiple MGRS tiles
S2_col = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', tile=['12TUL', '12TUM', '12TUN'])

#Rather than using tiles, we can use a boundary (ee.Geometry) or if using Sentinel-2 we can use relative orbits - here we use a relative orbit number which provides a full swath of data
S2_col_orbit_filter = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', relative_orbit_number=127)

#Here is an example of using a boundary to filter the collection - first defining a boundary, in this case the county of Salt Lake City
counties = ee.FeatureCollection('TIGER/2018/Counties')
salt_lake_county = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Salt Lake'),
    ee.Filter.eq('STATEFP', '49')))
salt_lake_geometry = salt_lake_county.geometry()

S2_col_boundary_filter = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-15', boundary=salt_lake_geometry)

#You can filter for clouds by setting the cloud percentage threshold - here we set it to 15%
S2_col_low_clouds = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', relative_orbit_number=127, cloud_percentage_threshold=15)

#You can also filter for images with a lot of NoData - which happens more than you'd think - here we set the threshold to 15% as well
S2_col_no_blank_images = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', relative_orbit_number=127, nodata_threshold=15)


In [4]:
print(S2_col.dates)

['2023-06-01', '2023-06-01', '2023-06-02', '2023-06-02', '2023-06-04', '2023-06-04', '2023-06-04', '2023-06-06', '2023-06-06', '2023-06-07', '2023-06-07', '2023-06-09', '2023-06-09', '2023-06-09', '2023-06-11', '2023-06-11', '2023-06-12', '2023-06-12', '2023-06-12', '2023-06-14', '2023-06-14', '2023-06-14', '2023-06-16', '2023-06-16', '2023-06-17', '2023-06-17', '2023-06-19', '2023-06-19', '2023-06-19', '2023-06-21', '2023-06-21', '2023-06-22', '2023-06-22', '2023-06-22', '2023-06-24', '2023-06-24', '2023-06-24', '2023-06-26', '2023-06-26', '2023-06-27', '2023-06-27', '2023-06-29', '2023-06-29', '2023-06-29']


In [5]:
# Similar examples for Landsat collections - showing how to filter using tiles or boundaries
col = LandsatCollection(start_date='2023-06-01', end_date='2023-06-30', tile_row=32, tile_path=38)
tile_filtered_col = LandsatCollection(start_date='2023-06-01', end_date='2023-06-30', tile_row=32, tile_path=38, cloud_percentage_threshold=50)
SLC_filtered_col = LandsatCollection(start_date='2023-06-01', end_date='2023-06-30', boundary=salt_lake_geometry, cloud_percentage_threshold=15)

In [6]:
# Showing how to print properties of an image collection 
# by turning the LandsatCollection object into an ee.ImageCollection object and using the .getInfo() parameter
print(SLC_filtered_col.collection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 167985, 0, -30, 4587315]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 167985, 0, -30, 4587315]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 167985, 0, -30, 4587315]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 167985, 0, -30, 4587315]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs':

When a collection is defined it is a Sentinel2Collection or LandsatCollection object - not an Earth Engine object - but it is easy to go back and forth!

In [7]:
#If we print the variable of one of the collections we see it is a Sentinel2Collection object
print(S2_col)

### This leads us to utilizing attributes and property attributes for general data management
- ### Attributes and property attributes automatically call on functions and specify default thresholds for easy processing of collections
- ### There are a plethora of property attributes available, and most of them are chainable to do multiple sets of processing with one line of code.
- ### Results from property attributes are cached as attributes to prevent processing something that has already been processed
#### SEE THE DOCUMENTATION FOR FULL LIST OF ATTRIBUTES AND PROPERTY ATTRIBUTES FOR THE AVAILABLE MODULES
#### Below are the useful attributes and property attributes for the Sentine2Collection class

    Attributes:
        collection (returns: ee.ImageCollection): Returns an ee.ImageCollection object from any Sentinel2Collection image collection object

    Property Attributes:
        dates_list (returns: Server-Side List): Unreadable Earth Engine list of image dates (server-side)
        
        dates (returns: Client-Side List): Readable pythonic list of image dates (client-side)
        
        masked_clouds_collection (returns: Sentinel2Collection image collection): Returns collection with clouds masked (transparent) for each image

        masked_to_water_collection (returns: Sentinel2Collection image collection): Returns collection masked to just water pixels

        masked_water_collection (returns: Sentinel2Collection image collection): Returns collection with water pixels masked
        
        max (returns: ee.Image): Returns a temporally reduced max image (calculates max at each pixel)
        
        median (returns: ee.Image): Returns a temporally reduced median image (calculates median at each pixel)
        
        mean (returns: ee.Image): Returns a temporally reduced mean image (calculates mean at each pixel)
        
        min (returns: ee.Image): Returns a temporally reduced min image (calculates min at each pixel)
        
        ndwi (returns: ee.ImageCollection): Returns Sentinel2Collection image collection of singleband NDWI (water) rasters
        
        ndvi (returns: ee.ImageCollection): Returns Sentinel2Collection image collection of singleband NDVI (vegetation) rasters

        halite (returns: ee.ImageCollection): Returns Sentinel2Collection image collection of singleband halite index rasters

        gypsum (returns: ee.ImageCollection): Returns Sentinel2Collection image collection of singleband gypsum index rasters

        turbidity (returns: ee.ImageCollection): Returns Sentinel2Collection image collection of singleband NDTI (turbidity) rasters

        chlorophyll (returns: ee.ImageCollection): Returns Sentinel2Collection image collection of singleband 2BDA (relative chlorophyll-a) rasters

        MosaicByDate (returns: Sentinel2Collection image collection): Mosaics image collection where images with the same date are mosaiced into the same image. Calculates total cloud percentage for subsequent filtering of cloudy mosaics.


In [8]:
# We can turn a Sentinel2Collection or LandsatCollection object into an Earth Engine image collection using the collection attribute
S2_gee_col = S2_col.collection
print('The collection is now a', type(S2_gee_col))

# Say you have an Earth Engine image collection object but you want to turn it into a Sentinel2Collection or LandsatCollection object, 
# just feed it in as a collection!
S2_col = Sentinel2Collection(collection=S2_gee_col)
print('The collection is back to a', type(S2_col))

# We can easily print the dates of all of the images in the collection using the dates attribute - this is a client-side operation
S2_dates = S2_col.dates
print('Readable list of image dates (client-side)', S2_dates)

# Alternatively, we can make a list of server-side dates for iteration, if needed
S2_dates_server_side = S2_col.dates_list
print('Server side dates are of type:', type(S2_dates_server_side))

# You can easily mask out clouds or water in the image collections
S2_masked_clouds = S2_col.masked_clouds_collection
S2_masked_to_water = S2_col.masked_to_water_collection

The collection is now a <class 'ee.imagecollection.ImageCollection'>
The collection is back to a <class 'RadGEEToolbox.Sentinel2Collection.Sentinel2Collection'>
Readable list of image dates (client-side) ['2023-06-01', '2023-06-01', '2023-06-02', '2023-06-02', '2023-06-04', '2023-06-04', '2023-06-04', '2023-06-06', '2023-06-06', '2023-06-07', '2023-06-07', '2023-06-09', '2023-06-09', '2023-06-09', '2023-06-11', '2023-06-11', '2023-06-12', '2023-06-12', '2023-06-12', '2023-06-14', '2023-06-14', '2023-06-14', '2023-06-16', '2023-06-16', '2023-06-17', '2023-06-17', '2023-06-19', '2023-06-19', '2023-06-19', '2023-06-21', '2023-06-21', '2023-06-22', '2023-06-22', '2023-06-22', '2023-06-24', '2023-06-24', '2023-06-24', '2023-06-26', '2023-06-26', '2023-06-27', '2023-06-27', '2023-06-29', '2023-06-29', '2023-06-29']
Server side dates are of type: <class 'ee.ee_list.List'>


### Utilizing methods for general data management
Many functions are available as methods to manage the data alongside attributes

    Methods:        
        ndwi_collection(self, threshold)
        
        ndvi_collection(self, threshold)
        
        halite_collection(self, threshold)
        
        gypsum_collection(self, threshold)

        turbidity_collection(self, threshold)

        chlorophyll_collection(self, threshold)

        get_filtered_collection(self)

        get_boundary_filtered_collection(self)

        get_orbit_filtered_collection(self)

        get_orbit_and_boundary_filtered_collection(self)
        
        mask_to_polygon(self, polygon)

        mask_out_polygon(self, polygon)

        masked_water_collection_NDWI(self, threshold)

        masked_to_water_collection_NDWI(self, threshold)
        
        mask_halite(self, threshold)
        
        mask_halite_and_gypsum(self, halite_threshold, gypsum_threshold)
        
        PixelAreaSumCollection(self, band_name, geometry, threshold, scale, maxPixels)

        image_grab(self, img_selector)
        
        custom_image_grab(self, img_col, img_selector)
        
        image_pick(self, img_date)
        
        CollectionStitch(self, img_col2)

In [9]:
# Here are some examples of utilizing method functions and attributes of the Sentinel2Collection and LandsatCollection classes

# Mask entire collection based on geometry
masked_S2_col = S2_col_orbit_filter.mask_to_polygon(salt_lake_geometry)

# Mosaic images in collection that share an image date
mosaiced_S2_col = S2_col_boundary_filter.MosaicByDate

# Mask water pixels from each single image using quality bands
water_masked_S2_col = S2_col_boundary_filter.masked_water_collection

# Mask water pixels from each single image using NDWI - where values less than the specified threshold are masked in each image
water_masked_S2_col = S2_col_boundary_filter.masked_water_collection_NDWI(threshold=0)

#### Example chaining of methods - where we first mosaic the collection, mask the collection to water pixels, then calculate relative turbidity for each image

In [10]:
# Example chaining of methods - where we first mosaic the collection, mask the collection to water pixels, then calculate relative turbidity for each image

turbidity_chain_example = S2_col.MosaicByDate.masked_to_water_collection.turbidity
print(turbidity_chain_example.dates)

['2023-06-01', '2023-06-02', '2023-06-04', '2023-06-06', '2023-06-07', '2023-06-09', '2023-06-11', '2023-06-12', '2023-06-14', '2023-06-16', '2023-06-17', '2023-06-19', '2023-06-21', '2023-06-22', '2023-06-24', '2023-06-26', '2023-06-27', '2023-06-29']


#### Arguably, one of the most useful method functions is image_grab() or image_pick() which allow you to iteratively select images from an image collection

In [11]:
# Select image from collection based on index
image_from_S2_collection = mosaiced_S2_col.image_grab(-1)

# Select image from collection based on date
image_from_S2_collection = mosaiced_S2_col.image_pick(mosaiced_S2_col.dates[-1])

print(image_from_S2_collection.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'int'

### Using static functions
Any of the static functions can be used on traditional earth engine image objects or mapped to EE collections. Below are the static methods for the Sentinel2Collection class.

    Static Methods:
        image_dater(image)
        
        sentinel_ndwi_fn(image, threshold)
        
        sentinel_ndvi_fn(image, threshold)
        
        sentinel_halite_fn(image, threshold)
        
        sentinel_gypsum_fn(image, threshold)

        sentinel_turbidity_fn(image, threshold)
        
        sentinel_chlorophyll_fn(image, threshold)
        
        MaskWaterS2(image)

        MaskToWaterS2(image)
        
        MaskWaterS2ByNDWI(image, threshold)

        MaskToWaterS2ByNDWI(image, threshold)

        halite_mask(image, threshold)
        
        gypsum_and_halite_mask(image, halite_threshold, gypsum_threshold)
        
        MaskCloudsS2(image)
        
        PixelAreaSum(image, band_name, geometry, threshold=-1, scale=30, maxPixels=1e12)

In [12]:
# adding image date properties to images (very general example)
S2_col_date_props = S2_col.collection.map(Sentinel2Collection.image_dater)

# masking water from images
S2_water_masked_col = S2_col.collection.map(Sentinel2Collection.MaskWaterS2)

# calculating the surface area of pixels of interest as square meters(water pixels from NDWI for example)
water_area = Sentinel2Collection.PixelAreaSum(image=mosaiced_S2_col.ndwi.image_grab(-1), band_name='ndwi', geometry=salt_lake_geometry, threshold=0, scale=10)

print('Square meters of water in image:', water_area.getInfo().get('properties')['ndwi'])

Square meters of water in image: 1647329756.0751777


In [17]:
# Showing how to make an image collection with pixel area calculated for all images in the collection (using ndwi images as example), and how to assess 
# the area calculations using aggregate_array()

area_col = mosaiced_S2_col.ndwi.PixelAreaSumCollection(band_name='ndwi', geometry=salt_lake_geometry, threshold=0, scale=50)
print('Square meters of water in images are:', area_col.aggregate_array('ndwi').getInfo())
print('Dates of images:', mosaiced_S2_col.dates)

Square meters of water in images are: [1069594768.6520188, 251369607.87758926, 247235936.65393806, 496807659.3774899, 192290154.215422, 1646335856.4432714]
Dates of images: ['2023-06-01', '2023-06-04', '2023-06-06', '2023-06-09', '2023-06-11', '2023-06-14']
